This is an example of a simple CNN developed, trained and utilized

AI was used to help generate the codebase

Note: Make sure that the tensorflow package is installed in your device.

In [1]:
# Lib imports
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, regularizers
import numpy as np

In [2]:
# DATASET DIRECTORY CONFIGURATION
# Download and unzip the dataset from Kaggle, set the directory paths accordingly.
train_dir = "muffin_vs_chihuahua/train"  # e.g. './muffin-vs-chihuahua/train'
test_dir = "muffin_vs_chihuahua/test"    # e.g. './muffin-vs-chihuahua/test'

In [3]:
# IMAGE PARAMETERS
# Used to resize the input images, also will determine the input size of your input layer.
IMG_SIZE = (128, 128)
BATCH_SIZE = 32

In [4]:
# DATA PREPROCESSING & AUGMENTATION
# Optional but recommended for image processing tasks, especially with limited data.
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    validation_split=0.2
)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)
val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

Found 3788 images belonging to 2 classes.
Found 945 images belonging to 2 classes.
Found 945 images belonging to 2 classes.
Found 1184 images belonging to 2 classes.
Found 1184 images belonging to 2 classes.


In [5]:
# IMPROVED CNN MODEL ARCHITECTURE WITH REGULARIZATION AND DROPOUT

# Some modifications are applied
initial_learning_rate = 0.001
# We are combining ExponentialDecay with Adam optimizer for better learning rate management
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=10000,
    decay_rate=0.9,
    staircase=True
)

# Create the optimizer with the learning rate schedule
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

# Applied dropout layers and L2 regularization to reduce overfitting
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.001), input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.25),
    layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.25),
    layers.Conv2D(128, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    layers.MaxPooling2D(2, 2),
    layers.Flatten(),
    layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

c:\Users\Acer\school\25-26\.venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [6]:
# Configure the model optimizers, loss function, and metrics
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy']) # old
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# TRAINING THE CNN
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)

Epoch 1/10


In [ ]:
# EVALUATE THE MODEL
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc}")

In [ ]:
# SAVE THE IMPROVED MODEL
model.save('exercise_6_trained_model_improved.keras')

In [ ]:
# SIMPLE INFERENCE SCRIPT
from tensorflow.keras.preprocessing import image

def predict_image(img_path, model_path='exercise_6_trained_model_improved.keras'):
    model = tf.keras.models.load_model(model_path)
    img = image.load_img(img_path, target_size=IMG_SIZE)
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    pred = model.predict(img_array)[0,0]
    label = "Chihuahua" if pred >= 0.5 else "Muffin"
    print(f"Prediction: {label} (confidence: {pred:.2f})")
    return label, pred


In [ ]:
# Example usage:
predict_image("muffin_vs_chihuahua/test/chihuahua/img_0_5.jpg")
predict_image("muffin_vs_chihuahua/test/chihuahua/img_0_8.jpg")

## EXERCISE 6 ANSWERS

### Improvements Applied:
1. **Dropout Layers**: Added dropout (0.25 after Conv layers, 0.5 after Dense layer) to prevent overfitting
2. **L2 Regularization**: Applied L2 regularization (0.001) to all Conv2D and Dense layers to penalize large weights

### Results:

In [ ]:
# a. Test Accuracy (5 points)
print("=" * 60)
print("ANSWER a: TEST ACCURACY")
print("=" * 60)
print(f"Test Accuracy: {test_acc:.4f} ({test_acc*100:.2f}%)")
print(f"Test Loss: {test_loss:.4f}")
print("=" * 60)

In [ ]:
# b. img_0_5 and img_0_8 predictions (10 points)
print("\n" + "=" * 60)
print("ANSWER b: IMAGE PREDICTIONS AND CONFIDENCE")
print("=" * 60)

# Predict img_0_5
print("\nImage: img_0_5.jpg")
label_5, conf_5 = predict_image("muffin_vs_chihuahua/test/chihuahua/img_0_5.jpg")

print("\nImage: img_0_8.jpg")
label_8, conf_8 = predict_image("muffin_vs_chihuahua/test/chihuahua/img_0_8.jpg")

print("\n" + "=" * 60)
print("SUMMARY:")
print(f"  img_0_5: {label_5} (confidence: {conf_5:.4f})")
print(f"  img_0_8: {label_8} (confidence: {conf_8:.4f})")
print("=" * 60)

In [ ]:
# c. Model saved as "exercise_6_trained_model_improved.h5" (10 points)
print("\n" + "=" * 60)
print("ANSWER c: MODEL SAVED")
print("=" * 60)
print("✓ Model saved as: exercise_6_trained_model_improved.h5")
print("✓ Model includes:")
print("  - Dropout layers (0.25 after Conv layers, 0.5 after Dense)")
print("  - L2 regularization (0.001) on all Conv2D and Dense layers")
print("=" * 60)